In [6]:
import sys, pathlib
sys.path.append(str(pathlib.Path.cwd().parent))
# sys.path

In [7]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from tabulate import tabulate

In [8]:
%load_ext autoreload
%autoreload 2
%aimport optiml.queries
%matplotlib inline

In [9]:
from optiml.connection import SnowflakeConnConfig
from optiml.queries import SNFLKQuery
connection = SnowflakeConnConfig(accountname='VUJSGPR-XG91870', warehousename="XSMALL_WH").create_connection()
qlib = SNFLKQuery(connection, 'KIV')

Connecting...


# Total cost breakdown

## Visualization

In [10]:
df = qlib.total_cost_breakdown('2022-10-01', '2022-10-31')
df

,cost_category,credits,dollars
0,Storage,0.000000,84.288657
1,Compute,1209.029722,2418.059444
2,Cloud Service,46.445960,92.891919
3,Autoclustering,0.000000,0.000000
4,Materialization Views,0.000000,0.000000
5,Replication,0.000000,0.000000
6,Search Optimization,0.000000,0.000000
7,Snowpipe,0.000000,0.000000


In [11]:
df.dtypes

cost_category     object
credits          float64
dollars          float64
dtype: object

In [35]:
fig = make_subplots(
    rows=1, cols=2,
    specs=[[{"type": "table"},{"type": "pie"}]]
)
# fig = go.Figure()

fig.add_trace(go.Pie(labels=df['cost_category'].tolist(), values=df['dollars'].tolist(),name="Dollars"),row=1,col=1)
fig.add_trace(go.Pie(labels=df['cost_category'].tolist(), values=df['credits'].tolist(),name='Credits'),row=1,col=2)

# fig.add_trace(go.Table(header=dict(values=['cost_category','credits', 'dollars'],font_size=10),
#                   cells=dict(values=[df['cost_category'],df['credits'],df['dollars']],font_size=10)),row=1, col=1)
#fig.update_layout(font_size=12)
fig.update_layout(
    title={
        'text': "Breakdown of total cost by usage category",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})
fig.show()

## Analysis
- Compute forms overwhelming majority of the cost
- Cloud services are in aggregate < 10 % of compute so are not an added expense
- Autoclustering used to be on in 2020 but isn't anymore
- Should explore if there are any compute efficiencies to be gained by 
    - Autoclustering
    - Materialized views / caching
    - Search optimization
    - Query optimizations / reducing scans

* Largely used by compute
* In 2020 auto clustering was on but was turned off - why?
* Storage credits at 0 since storage is charged as a flat rate - are numbers consistent?
* Definitely did not use $80K worth of credits in October since total bill is $50K - so something is wrong!


In [27]:
df = qlib.cost_of_compute_ts('2022-10-12', '2022-10-13')

In [28]:
df

,WAREHOUSE_NAME,CREDITS,DOLLARS,START_TIME,END_TIME
0,ML_WH,0.497222,0.994444,2022-10-12 00:00:00-07:00,2022-10-12 01:00:00-07:00
1,PROD_WH,2.000000,4.000000,2022-10-12 00:00:00-07:00,2022-10-12 01:00:00-07:00
2,DEV_WH,1.000000,2.000000,2022-10-12 00:00:00-07:00,2022-10-12 01:00:00-07:00
3,DEV_WH,1.000000,2.000000,2022-10-12 01:00:00-07:00,2022-10-12 02:00:00-07:00
4,PROD_WH,2.000000,4.000000,2022-10-12 01:00:00-07:00,2022-10-12 02:00:00-07:00
...,...,...,...,...,...
59,DAILY_REFRESH_WH,1.133611,2.267222,2022-10-12 16:00:00-07:00,2022-10-12 17:00:00-07:00
60,DEV_WH,1.000000,2.000000,2022-10-12 17:00:00-07:00,2022-10-12 18:00:00-07:00
61,DAILY_REFRESH_WH,1.000000,2.000000,2022-10-12 17:00:00-07:00,2022-10-12 18:00:00-07:00
62,ML_WH,0.106389,0.212778,2022-10-12 17:00:00-07:00,2022-10-12 18:00:00-07:00


In [14]:
df['TOTAL_CREDITS_USED'].sum()

Decimal('246.493055550')